## Dataset

In [1]:
# mount drive for access to the files
from google.colab import drive

drive.mount("/content/drive")

# all the drive the files are present in "/content/drive/My Drive"
!ls "/content/drive/My Drive/Beuth Uni/Master Thesis"

import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Data
'Data Quality in ML Production Systems.pdf'
'Datawig: Missing Value Imputation for Tables.pdf'
 Declaration
 Images
 jenga
 jenga.pdf
 MICE_Multivariate_Imputation_by_Chained_Equations_.pdf


In [2]:
!pip install openml
!pip install pyod

!pip install mxnet autogluon
!pip install mxnet-mkl --pre --upgrade

Requirement already up-to-date: mxnet-mkl in /usr/local/lib/python3.6/dist-packages (1.6.0)


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from jenga.basis import Dataset

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ModuleNotFoundError: ignored

In [3]:
seed = 10

In [5]:
dataset = Dataset(seed, "credit-g")

all_data = dataset.all_data
attribute_names = dataset.attribute_names
attribute_types = dataset.attribute_types

categorical_columns = dataset.categorical_columns
numerical_columns = dataset.numerical_columns

print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")

Dataset: credit-g
Found 13 categorical and 7 numeric features 



### Visualize the dataset

In [ ]:
## plot the original dataset
def hide_current_axis(*args, **kwds):
        plt.gca().set_visible(False)
        
def plot_data(data):
    sns.set_style("white") # grid/no grid style: darkgrid, whitegrid, dark, white, ticks
    
    plot = sns.pairplot(data, hue="class")
    plot.map_upper(hide_current_axis)
    plt.show()

In [ ]:
plot_data(all_data)

### Get training and test sets

In [6]:
train_data, train_labels, test_data, test_labels = dataset.get_train_test_data()

## Model

### Model using AutoGluon

In [ ]:
import autogluon as ag
from autogluon import TabularPrediction as task

label_col = 'class'

tr_data = train_data
tr_data[label_col] = train_labels

model = task.fit(train_data=tr_data, label=label_col)

# predictions
y_pred = model.predict(test_data)

# predictor performance
perf = model.evaluate_predictions(y_true=test_labels, y_pred=y_pred, auxiliary_metrics=True)

ModuleNotFoundError: ignored

In [ ]:
import random
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import autogluon as ag
from autogluon import TabularPrediction as task


class Model:

  def __init__(self, seed, train_data, train_labels, test_data, test_labels, pipeline, learner, param_grid):

    ## fix random seeds for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    
    ## train and test data and labels
    self.train_data = train_data
    self.train_labels = train_labels
    self.test_data = test_data
    self.test_labels = test_labels
    
    ## preprocessing pipeline
    self.pipeline = pipeline
    
    ## information for model parameters
    self.learner = learner
    self.param_grid = param_grid
    
  
  def __repr__(self):
    return f"{self.__class__.__name__}: {self.__dict__}"
  
  
  # method for training a model on the raw data with preprocessing
  def fit_model(self):

    if self.learner == 'autogluon':
      label_col = 'class'
      
      tr_data = self.train_data
      tr_data[label_col] = self.train_labels
      
      model = task.fit(train_data=tr_data, label=label_col)

      return model
    else:
      grid_search = GridSearchCV(self.pipeline, self.param_grid, scoring='roc_auc', cv=5, verbose=1, n_jobs=-1)
      model = grid_search.fit(self.train_data, self.train_labels)

      return model

  # method for computing evaluation scores
  def eval_scores(self, model):
    if self.learner == 'autogluon':
      # predictions
      y_pred = model.predict(self.test_data)
      
      # predictor performance
      perf = model.evaluate_predictions(y_true=self.test_labels, y_pred=y_pred, auxiliary_metrics=True)
      
      return perf
    else:
      pred_prob = model.predict_proba(self.test_data)
      roc_auc_acore = roc_auc_score(self.test_labels, np.transpose(pred_prob)[1])
      
      return roc_auc_acore
    

In [ ]:
## define preprocessing pipeline if not given
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

def get_pipeline(learner):
  # preprocessing pipeline for numerical columns
  transformer_numeric = Pipeline([
      ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
      ('standard_scale', StandardScaler())
  ])

  # preprocessing pipeline for categorical columns
  transformer_categorical = Pipeline([
      ('imputer', SimpleImputer(strategy='constant', fill_value='__NA__')),
      ('one_hot_encode', OneHotEncoder(handle_unknown='ignore'))
  ])

  # preprocessor
  feature_transform = ColumnTransformer(transformers=[
      ('categorical_features', transformer_categorical, categorical_columns),
      ('numerical_features', transformer_numeric, numerical_columns)
  ])

  ## prediction pipeline: append classifier (learner) to the preprocessing pipeline
  pipeline = Pipeline([
      ('features', feature_transform),
      ('learner', learner)
  ])

  return pipeline

In [ ]:
# test autogluon
# pipeline in the case of autogluon is not used so anything is given
model_obj = Model(seed, train_data, train_labels, test_data, test_labels, get_pipeline(SGDClassifier(loss='log')), learner="autogluon", param_grid={})
model = model_obj.fit_model()
scores = model_obj.eval_scores(model)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200720_140007/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200720_140007/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 21
Preprocessing data ...
Here are the 2 unique label values in your data:  ['good', 'bad']
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Selected class <--> label mapping:  class 1 = bad, class 0 = good
Train Data Class Count: 2
Feature Generator processed 800 data points with 20 features
Original Features (raw dtypes):
	category features: 13
	float64 features: 7
Original Features (inferred dtypes):
	category features: 13
	float features: 7
Generated Features (special dtypes):
Final Features (raw dtypes):
	category feat

In [ ]:
## model parameters
## models is a dict where key = leaner & value = param_grid
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

models = {SGDClassifier(loss='log'): {'learner__max_iter': [500, 1000, 5000], 
                                         'learner__penalty': ['l2', 'l1', 'elasticnet'], 
                                         'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
                                        }, 
          RandomForestClassifier():{'learner__n_estimators': [100, 200, 500], 
                                    'learner__max_depth': [5, 10, 15]
                                   },
          "autogluon": {}
         }

In [ ]:
for learner, param_grid in models.items():
  print(learner, param_grid)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False) {'learner__max_iter': [500, 1000, 5000], 'learner__penalty': ['l2', 'l1', 'elasticnet'], 'learner__alpha': [0.0001, 0.001, 0.01, 0.1]}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                    

In [ ]:
model_summary = []
scores_summary = []

for learner, param_grid in models.items():
  pipeline = None
  
  if learner == 'autogluon':
    pipeline = get_pipeline(SGDClassifier(loss='log'))
  else:
    pipeline = get_pipeline(learner)
  
  model_obj = Model(seed, train_data, train_labels, test_data, test_labels, pipeline, learner, param_grid)
  
  model = model_obj.fit_model()
  model_summary.append(model)

  scores = model_obj.eval_scores(model)
  scores_summary.append(scores)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 177 out of 180 | elapsed:    8.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   19.5s finished
No output_directory specified. Models will be saved in: AutogluonModels/ag-20200720_140922/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200720_140922/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 21
Preprocessing data ...
Here are the 2 unique label values in your data:  ['bad', 'good']
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Selected class <--> label mapping:  class 1 = good, class 0 = bad
Train Data Class Count: 2
NumExpr defaulting to 2 threads.
Feature Generator processed 800 data points with 20 features
Original Features (raw dtypes):
	category features: 13
	float64 features: 7
Original Features (inferred dtypes):
	category features

In [ ]:
model_summary

[GridSearchCV(cv=5, error_score=nan,
              estimator=Pipeline(memory=None,
                                 steps=[('features',
                                         ColumnTransformer(n_jobs=None,
                                                           remainder='drop',
                                                           sparse_threshold=0.3,
                                                           transformer_weights=None,
                                                           transformers=[('categorical_features',
                                                                          Pipeline(memory=None,
                                                                                   steps=[('imputer',
                                                                                           SimpleImputer(add_indicator=False,
                                                                                                         copy=True,
           

In [ ]:
scores_summary

[0.8093735390369332,
 0.7849462365591399,
 OrderedDict([('accuracy', 0.73),
              ('accuracy_score', 0.73),
              ('balanced_accuracy_score', 0.6666666666666667),
              ('matthews_corrcoef', 0.3472488574259035),
              ('f1_score', 0.7299999999999999),
              ('classification_report',
               {'accuracy': 0.73,
                'bad': {'f1-score': 0.5344827586206897,
                 'precision': 0.5740740740740741,
                 'recall': 0.5,
                 'support': 62},
                'good': {'f1-score': 0.8098591549295775,
                 'precision': 0.7876712328767124,
                 'recall': 0.8333333333333334,
                 'support': 138},
                'macro avg': {'f1-score': 0.6721709567751336,
                 'precision': 0.6808726534753933,
                 'recall': 0.6666666666666667,
                 'support': 200},
                'weighted avg': {'f1-score': 0.7244924720738223,
                 'precisi

## Corruptions

In [ ]:
from jenga.corruptions.perturbations import Perturbation

In [ ]:
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise

In [ ]:
corruptions = [MissingValues, SwappedValues, Scaling, GaussianNoise]
fraction = 0.5

In [ ]:
# corruption perturbations to apply
corr_perturbations = Perturbation(categorical_columns, numerical_columns)

In [ ]:
test_data_corrupted, perturbations, cols_perturbed, summary_col_corrupt = corr_perturbations.apply_perturbation(test_data, corruptions, fraction)

Applying perturbations... 

MissingValues: {'column': 'other_parties', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MAR'}
SwappedValues: {'column_a': 'checking_status', 'column_b': 'employment', 'fraction': 0.5}
Scaling: {'column': 'residence_since', 'fraction': 0.5}
GaussianNoise: {'column': 'age', 'fraction': 0.5}


### Visualize the original and corrupted test set

In [ ]:
## original test data
plot_data(pd.concat([test_data, pd.Series(test_labels, name='class')], axis=1))

In [ ]:
## corrupted test data
plot_data(pd.concat([test_data_corrupted, pd.Series(test_labels, name='class')], axis=1))

## Model & Corruptions using PPP

In [7]:
from sklearn.linear_model import SGDClassifier

learner = SGDClassifier(loss='log')
param_grid = {
    'learner__max_iter': [500, 1000, 5000],
    'learner__penalty': ['l2', 'l1', 'elasticnet'], 
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

In [8]:
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise

corruptions = [MissingValues, SwappedValues, Scaling, GaussianNoise]
fraction = 0.5
num_repetitions = 5

In [9]:
from jenga.cleaning.ppp import PipelinePerformancePrediction

ppp = PipelinePerformancePrediction(seed, train_data, train_labels, test_data, test_labels, categorical_columns, numerical_columns, learner, param_grid)
ppp_model = ppp.fit_ppp(train_data)

## generate corrpted data
df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(test_data, corruptions, fraction, num_repetitions)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    4.3s



Generating corrupted training data on 200 rows... 

	perturbation: MissingValues: {'column': 'credit_amount', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: SwappedValues: {'column_a': 'other_parties', 'column_b': 'property_magnitude', 'fraction': 0.5}
	perturbation: Scaling: {'column': 'existing_credits', 'fraction': 0.5}
	perturbation: GaussianNoise: {'column': 'residence_since', 'fraction': 0.5}
	perturbation: MissingValues: {'column': 'foreign_worker', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: SwappedValues: {'column_a': 'purpose', 'column_b': 'savings_status', 'fraction': 0.5}
	perturbation: Scaling: {'column': 'residence_since', 'fraction': 0.5}
	perturbation: GaussianNoise: {'column': 'existing_credits', 'fraction': 0.5}
	perturbation: MissingValues: {'column': 'savings_status', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: SwappedValues: {'column_a': 'other_payment_plans', 'column_b': 'job', 'fract

[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    9.9s finished


## Cleaning

### Cleaning using AutoGluon

In [10]:
from autogluon import TabularPrediction as task

In [11]:
def cat_cols_to_str(df):
  for col in df.columns:
    if pd.api.types.is_categorical_dtype(df[col]):
      df[col] = df[col].astype(str)

  return df

In [12]:
train_data = cat_cols_to_str(train_data)
df_corrupted = cat_cols_to_str(df_corrupted)
test_data = cat_cols_to_str(test_data)

In [13]:
categorical_precision_threshold = 0.7
numerical_std_error_threshold = 0.7

#### Training

In [ ]:
predictors = {}
predictable_cols = []

In [ ]:
for col in categorical_columns:
  predictors[col] = task.fit(train_data=train_data, label=col, problem_type='multiclass')

  y_test = test_data[col].dropna() # take only the non-nan records # test_data? OR split the train_data again into train and test
  y_pred = predictors[col].predict(test_data.drop([col], axis=1)) # drop the actual column before predicting

  perf = predictors[col].evaluate_predictions(y_test, y_pred, auxiliary_metrics=True)

  labels = [k for k in perf['classification_report'].keys() if k not in ['accuracy', 'macro avg', 'weighted avg']]

  high_precision_labels = []
  for label in labels:
    if perf['classification_report'][label]['precision'] > categorical_precision_threshold:
      high_precision_labels.append(label)

  if high_precision_labels:
    predictable_cols.append(col)
    predictors[col].high_precision_labels = high_precision_labels


No output_directory specified. Models will be saved in: AutogluonModels/ag-20200723_225521/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200723_225521/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 20
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 800 data points with 19 features
Original Features (raw dtypes):
	float64 features: 7
	object features: 12
Original Features (inferred dtypes):
	float features: 7
	object features: 12
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 7
	category features: 12
Final Features:
	float features: 7
	category features: 12
	Data preprocessing and feature engineering runtime = 0.13s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Fitting model: RandomForestClassifierGini

In [ ]:
for col in numerical_columns:
  predictors[col] = task.fit(train_data=train_data, label=col, problem_type='regression')

  y_test = test_data[col].dropna() # take only the non-nan records # test_data? OR split the train_data again into train and test
  y_pred = predictors[col].predict(test_data.drop([col], axis=1)) # drop the actual column before predicting

  perf = predictors[col].evaluate_predictions(y_test, y_pred, auxiliary_metrics=True)

  if perf['root_mean_squared_error'] < numerical_std_error_threshold * y_test.std():
    predictable_cols.append(col)
    predictors[col].root_mean_squared_error = perf['root_mean_squared_error']

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200723_230205/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200723_230205/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 20
Preprocessing data ...
Feature Generator processed 800 data points with 19 features
Original Features (raw dtypes):
	object features: 13
	float64 features: 6
Original Features (inferred dtypes):
	object features: 13
	float features: 6
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 6
	category features: 13
Final Features:
	float features: 6
	category features: 13
	Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: root_mean_squared_error
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: root_mean_squared_error
Fitting model: RandomForestRegressorM

In [ ]:
print(f"Found {len(predictable_cols)} predictable columns: {predictable_cols}")

Found 10 predictable columns: ['credit_history', 'employment', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']


#### Outlier Detection/Removal

#### Imputation

In [14]:
from autogluon import TabularPrediction as task

In [15]:
def cat_cols_to_str(df):
  for col in df.columns:
    if pd.api.types.is_categorical_dtype(df[col]):
      df[col] = df[col].astype(str)

  return df

In [16]:
train_data = cat_cols_to_str(train_data)
df_corrupted = cat_cols_to_str(df_corrupted)

In [17]:
predictors = {}

for col in categorical_columns:
  predictors[col] = task.fit(train_data=train_data, label=col, problem_type='multiclass')
  
for col in numerical_columns:
  predictors[col] = task.fit(train_data=train_data, label=col, problem_type='regression')

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200724_110758/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200724_110758/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 20
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 800 data points with 19 features
Original Features (raw dtypes):
	float64 features: 7
	object features: 12
Original Features (inferred dtypes):
	float features: 7
	object features: 12
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 7
	category features: 12
Final Features:
	float features: 7
	category features: 12
	Data preprocessing and feature engineering runtime = 0.17s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Fitting model: RandomForestClassifierGini

In [18]:
df_imputed = df_corrupted.copy()

In [22]:
for col in df_corrupted.columns:
  df_imputed[col + '_imputed'] = predictors[col].predict(df_imputed.drop([col], axis=1)) # drop the actual column before predicting
  perf = predictors[col].evaluate_predictions(df_imputed[col], df_imputed[col + '_imputed'], auxiliary_metrics=True)

  df_imputed[col].fillna(df_imputed[col + '_imputed'], inplace=True)

Evaluation: accuracy on test data: 0.27
Evaluations on test data:
{
    "accuracy": 0.27,
    "accuracy_score": 0.27,
    "balanced_accuracy_score": 0.33604625575516833,
    "matthews_corrcoef": 0.08617079441077244
}
Detailed (per-class) classification report:
{
    "0<=X<200": {
        "precision": 0.2962962962962963,
        "recall": 0.1509433962264151,
        "f1-score": 0.2,
        "support": 53
    },
    "<0": {
        "precision": 0.5294117647058824,
        "recall": 0.16363636363636364,
        "f1-score": 0.24999999999999997,
        "support": 55
    },
    ">=200": {
        "precision": 0.11458333333333333,
        "recall": 0.6875,
        "f1-score": 0.1964285714285714,
        "support": 16
    },
    "no checking": {
        "precision": 0.43333333333333335,
        "recall": 0.34210526315789475,
        "f1-score": 0.3823529411764707,
        "support": 76
    },
    "accuracy": 0.27,
    "macro avg": {
        "precision": 0.34340618191721134,
        "recall": 

In [23]:
df_imputed

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,checking_status_imputed,duration_imputed,credit_history_imputed,purpose_imputed,credit_amount_imputed,savings_status_imputed,employment_imputed,installment_commitment_imputed,personal_status_imputed,other_parties_imputed,residence_since_imputed,property_magnitude_imputed,age_imputed,other_payment_plans_imputed,housing_imputed,existing_credits_imputed,job_imputed,num_dependents_imputed,own_telephone_imputed,foreign_worker_imputed
841,no checking,21.0,delayed previously,used car,2993.0,<100,1<=X<4,3.0,male single,none,200.0,real estate,28.000000,stores,own,2.0,unskilled resident,1.0,none,yes,>=200,28.264215,existing paid,radio/tv,3721.044223,<100,>=7,3.245162,female div/dep/mar,none,2.213055,life insurance,60.780865,none,own,1.895354,skilled,1.177597,none,yes
956,>=200,30.0,critical/other existing credit,radio/tv,3656.0,<100,>=7,4.0,no known savings,none,400.0,life insurance,-1.010227,stores,own,2.0,unskilled resident,1.0,none,yes,>=200,37.096821,all paid,furniture/equipment,3430.557236,<100,>=7,2.995577,female div/dep/mar,none,3.235311,car,62.912970,none,own,2.059008,skilled,1.060069,none,yes
544,no checking,12.0,critical/other existing credit,new car,1255.0,<100,>=7,4.0,male single,none,4.0,real estate,78.803083,none,own,2.0,unskilled resident,1.0,none,yes,<0,14.126552,critical/other existing credit,radio/tv,1665.526738,<100,>=7,3.052612,male single,none,3.636690,real estate,42.419790,none,own,1.900729,skilled,1.137137,none,yes
173,0<=X<200,8.0,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,<100,guarantor,200.0,real estate,33.000000,none,own,1.0,skilled,1.0,none,no,>=200,26.570163,existing paid,new car,983.597519,<100,>=7,3.072927,female div/dep/mar,none,2.042717,real estate,58.546811,none,own,1.442285,skilled,1.170546,none,yes
759,<0,12.0,critical/other existing credit,new car,691.0,<100,>=7,4.0,female div/dep/mar,none,300.0,life insurance,35.000000,none,own,2.0,skilled,1.0,none,yes,>=200,21.440085,existing paid,furniture/equipment,1215.041510,<100,>=7,3.851265,female div/dep/mar,none,3.408381,car,59.923627,none,own,2.034203,skilled,1.085467,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,<0,30.0,existing paid,repairs,11998.0,male div/sep,<1,1.0,<100,none,1.0,no known property,34.000000,none,own,1.0,unskilled resident,1.0,yes,yes,0<=X<200,36.570619,existing paid,new car,5056.321413,<100,1<=X<4,2.094082,male single,none,2.562646,life insurance,33.405687,none,for free,1.207713,skilled,1.132460,yes,yes
192,0<=X<200,27.0,existing paid,business,3915.0,male single,1<=X<4,4.0,<100,none,200.0,car,47.273307,none,own,1.0,skilled,2.0,yes,yes,>=200,39.343019,existing paid,new car,2952.729854,<100,>=7,2.874499,female div/dep/mar,none,2.658189,car,60.370933,none,own,1.594232,skilled,1.191276,yes,yes
398,0<=X<200,12.0,existing paid,new car,1223.0,male div/sep,>=7,1.0,<100,none,100.0,real estate,46.000000,none,rent,2.0,skilled,1.0,none,yes,>=200,20.026831,existing paid,new car,2577.824664,<100,>=7,3.343817,female div/dep/mar,none,3.479588,real estate,57.097446,none,own,1.441971,skilled,1.134272,none,yes
450,no checking,36.0,critical/other existing credit,used car,11054.0,no known savings,1<=X<4,4.0,male single,none,2.0,car,30.000000,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,no checking,43.865086,existing paid,used car,6446.838579,<100,1<=X<4,2.849027,male single,none,2.430758,car,37.722556,none,own,1.824590,skilled,1.078409,yes,yes


In [24]:
df_imputed = df_imputed[df_corrupted.columns]
df_imputed

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.0,delayed previously,used car,2993.0,<100,1<=X<4,3.0,male single,none,200.0,real estate,28.000000,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.0,critical/other existing credit,radio/tv,3656.0,<100,>=7,4.0,no known savings,none,400.0,life insurance,-1.010227,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,12.0,critical/other existing credit,new car,1255.0,<100,>=7,4.0,male single,none,4.0,real estate,78.803083,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,8.0,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,<100,guarantor,200.0,real estate,33.000000,none,own,1.0,skilled,1.0,none,no
759,<0,12.0,critical/other existing credit,new car,691.0,<100,>=7,4.0,female div/dep/mar,none,300.0,life insurance,35.000000,none,own,2.0,skilled,1.0,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,<0,30.0,existing paid,repairs,11998.0,male div/sep,<1,1.0,<100,none,1.0,no known property,34.000000,none,own,1.0,unskilled resident,1.0,yes,yes
192,0<=X<200,27.0,existing paid,business,3915.0,male single,1<=X<4,4.0,<100,none,200.0,car,47.273307,none,own,1.0,skilled,2.0,yes,yes
398,0<=X<200,12.0,existing paid,new car,1223.0,male div/sep,>=7,1.0,<100,none,100.0,real estate,46.000000,none,rent,2.0,skilled,1.0,none,yes
450,no checking,36.0,critical/other existing credit,used car,11054.0,no known savings,1<=X<4,4.0,male single,none,2.0,car,30.000000,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes


### Using class cleaner

In [10]:
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNN, PyODIsolationForest
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, AutoGluonImputation

cleaners = [
    # (NoOutlierDetection, MeanModeImputation),
    (NoOutlierDetection, AutoGluonImputation)
    # (PyODKNN, NoImputation),
    # (PyODKNN, MeanModeImputation),
    # (PyODKNN, AutoGluonImputation),
    # (PyODIsolationForest, NoImputation),
    # (PyODIsolationForest, MeanModeImputation),
    # (PyODIsolationForest, AutoGluonImputation)
]

In [11]:
from jenga.cleaning.clean import Clean

clean = Clean(train_data, df_corrupted, categorical_columns, numerical_columns, ppp, ppp_model, cleaners)
df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(train_data, df_corrupted)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200724_112422/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200724_112422/
AutoGluon Version:  0.0.12
Train Data Rows:    800
Train Data Columns: 20
Preprocessing data ...
Train Data Class Count: 4



Applying cleaners... 

PPP score no cleaning: {'roc_auc_acore': 0.5883590462833099, 'classification_report': {'bad': {'precision': 0.6363636363636364, 'recall': 0.22580645161290322, 'f1-score': 0.3333333333333333, 'support': 62}, 'good': {'precision': 0.7303370786516854, 'recall': 0.9420289855072463, 'f1-score': 0.8227848101265822, 'support': 138}, 'accuracy': 0.72, 'macro avg': {'precision': 0.6833503575076609, 'recall': 0.5839177185600748, 'f1-score': 0.5780590717299577, 'support': 200}, 'weighted avg': {'precision': 0.7012053115423902, 'recall': 0.72, 'f1-score': 0.6710548523206751, 'support': 200}}}
PPP scores with cleaning: 


Feature Generator processed 800 data points with 19 features
Original Features (raw dtypes):
	float64 features: 7
	object features: 12
Original Features (inferred dtypes):
	float features: 7
	object features: 12
Generated Features (special dtypes):
Final Features (raw dtypes):
	float features: 7
	category features: 12
Final Features:
	float features: 7
	category features: 12
	Data preprocessing and feature engineering runtime = 0.34s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Fitting model: RandomForestClassifierGini ...
	0.4625	 = Validation accuracy score
	0.97s	 = Training runtime
	0.22s	 = Validation runtime
Fitting model: RandomForestClassifierEntr ...
	0.4688	 = Validation accuracy score
	1.05s	 = Training runtime
	0.22s	 = Validation runtime
Fitting model: ExtraTreesClassifierGini ...
	0.5062	 = Validation accuracy sc

TypeError: ignored

In [1]:
df_cleaned

NameError: ignored

### Imputation

In [ ]:
mean_mode_imputer = MeanModeImputation(train_data, test_data_corrupted, categorical_columns, numerical_columns)

test_data_mm_imputed = mean_mode_imputer.fit_transform(train_data, test_data_corrupted)
test_data_mm_imputed

In [ ]:
datawig_imputer = DatawigImputation(train_data, test_data_corrupted, categorical_columns, numerical_columms)

test_data_dw_imputed = datawig_imputer.fit_transform(train_data, test_data_corrupted)
test_data_dw_imputed

##### Using PPP

In [ ]:
# for all imputers return scores, take best
# using ppp

In [ ]:
from jenga.cleaning.imputation import MeanModeImputation, DatawigImputation
from jenga.cleaning.ppp import PipelinePerformancePrediction

In [ ]:
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

In [ ]:
ppp = PipelinePerformancePrediction(seed, train_data, train_labels, test_data, test_labels, categorical_columns, numerical_columns, learner, param_grid)

In [ ]:
# generate corrpted test data
test_data_corrupted, perturbations, cols_perturbed = ppp.get_corrupted(test_data)

In [ ]:
imputer_candidates = [MeanModeImputation, DatawigImputation]

In [ ]:
imputers = []
for imputer in imputer_candidates:
    imputers.append(imputer(train_data, test_data_corrupted, categorical_columns, numerical_columns))

In [ ]:
imputers

In [ ]:
ppp_model = ppp.fit_ppp(train_data)

In [ ]:
score_no_cleaning = ppp.predict_score_ppp(ppp_model, test_data)
score_no_cleaning

In [ ]:
imputed_scores_ppp = []
for imputer in imputers:
    test_data_imputed = imputer.fit_transform(train_data, test_data_corrupted)
    imputed_score = ppp.predict_score_ppp(ppp_model, test_data_imputed)
    print(f"PPP score with {imputer}: {imputed_score}")
    imputed_scores_ppp.append(imputed_score)

In [ ]:
imputed_scores_ppp

##### Using PPP and Cleaner classes

In [ ]:
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.cleaner import Cleaner
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNN, PyODIsolationForest
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, DatawigImputation
from jenga.cleaning.clean import Clean

In [ ]:
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

In [ ]:
ppp = PipelinePerformancePrediction(seed, train_data, train_labels, test_data, test_labels, categorical_columns, numerical_columns, learner, param_grid)

In [ ]:
ppp_model = ppp.fit_ppp(train_data)

In [ ]:
ppp_model_score = ppp.predict_score_ppp(ppp_model, test_data)
ppp_model_score

In [ ]:
# generate corrpted test data
test_data_corrupted, perturbations, cols_perturbed = ppp.get_corrupted(test_data)

In [ ]:
score_no_cleaning = ppp.predict_score_ppp(ppp_model, test_data_corrupted)
score_no_cleaning

In [ ]:
cleaner_candidates = [
    (NoOutlierDetection, NoImputation),
    (NoOutlierDetection, MeanModeImputation),
    (NoOutlierDetection, DatawigImputation),
    (PyODKNN, NoImputation),
    (PyODKNN, MeanModeImputation),
    (PyODKNN, DatawigImputation),
    (PyODIsolationForest, NoImputation),
    (PyODIsolationForest, MeanModeImputation),
    (PyODIsolationForest, DatawigImputation)
]

In [ ]:
cleaners = []
for outd, imp in cleaner_candidates:
    cleaners.append(Cleaner(train_data, 
                            test_data_corrupted, 
                            categorical_columns, 
                            numerical_columns, 
                            outlier_detection = outd(train_data, 
                                                     test_data_corrupted, 
                                                     categorical_columns, 
                                                     numerical_columns), 
                            imputation = imp(train_data, 
                                             test_data_corrupted, 
                                             categorical_columns, 
                                             numerical_columns)
                           ))

In [ ]:
cleaner_scores_ppp = []
for cleaner in cleaners:
    test_data_cleaned = cleaner.apply_cleaner(train_data, test_data_corrupted, categorical_columns, numerical_columns)
    cleaner_score = ppp.predict_score_ppp(ppp_model, test_data_cleaned)
    print(f"PPP score with {cleaner}: {cleaner_score}")
    cleaner_scores_ppp.append(cleaner_score)

In [ ]:
cleaner_scores_ppp

In [ ]:
best_cleaning_idx = pd.Series(cleaner_scores_ppp).idxmax()
best_cleaning_idx

In [ ]:
best_cleaning_score = cleaner_scores_ppp[best_cleaning_idx]
best_cleaning_score

In [ ]:
if best_cleaning_score > score_no_cleaning:
    test_data_cleaned = cleaners[best_cleaning_idx].apply_cleaner(train_data, test_data_corrupted, categorical_columns, numerical_columns)
    print(f"Best cleaning method: {cleaners[best_cleaning_idx]}: {best_cleaning_score}")
else:
    print("Cleaning didnt't improve the score")

In [ ]:
## using clean class

In [ ]:
clean = Clean(train_data, test_data_corrupted, categorical_columns, numerical_columns, ppp, ppp_model)

In [ ]:
test_data_cleaned, score_no_cleaning, cleaner_scores_ppp = clean(train_data, test_data_corrupted)

In [ ]:
from jenga.cleaning.outlier_detection import NoOutlierDetection
from jenga.cleaning.imputation import NoImputation


class Cleaner:
    
    def __init__(self, 
                 df_train,
                 df_corrupted,
                 categorical_columns,
                 numerical_columns,
                 outlier_detection=NoOutlierDetection, 
                 imputation=NoImputation):
        self.outlier_detection = outlier_detection
        self.imputation = imputation
        
    
    def apply_cleaner(self, df_train, df_corrupted, categorical_columns, numerical_columns):
        df_cleaned = self.outlier_detection(df_train, df_corrupted)
        
        # do something for fixing/removing the outliers
        if 'outlier' in df_cleaned.columns:
            ### TODO 
            df_cleaned = df_cleaned.drop('outlier', axis=1)
            
        # impute
        df_cleaned = self.imputation(df_train, df_cleaned)
        
        return df_cleaned

In [ ]:
import pandas as pd

from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.cleaner import Cleaner
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNN, PyODIsolationForest
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, DatawigImputation


DEFAULT_CLEANERS = [
    (NoOutlierDetection, NoImputation),
    (NoOutlierDetection, MeanModeImputation),
    (NoOutlierDetection, DatawigImputation),
    (PyODKNN, NoImputation),
    (PyODKNN, MeanModeImputation),
    (PyODKNN, DatawigImputation),
    (PyODIsolationForest, NoImputation),
    (PyODIsolationForest, MeanModeImputation),
    (PyODIsolationForest, DatawigImputation)
]


class Clean:
    
    def __init__(self, 
                 df_train, 
                 df_corrupted, 
                 categorical_columns, 
                 numerical_columns,
                 ppp,
                 ppp_model,
                 cleaners=DEFAULT_CLEANERS):

        self.categorical_columns = categorical_columns
        self.numerical_columns = numerical_columns
        
        self.ppp = ppp
        self.ppp_model = ppp_model
        
        self.cleaners = []
        for outd, imp in cleaners:
            self.cleaners.append(Cleaner(df_train,
                                         df_corrupted,
                                         self.categorical_columns,
                                         self.numerical_columns,
                                         outlier_detection = outd(df_train,
                                                                  df_corrupted,
                                                                  self.categorical_columns,
                                                                  self.numerical_columns),
                                         imputation = imp(df_train,
                                                          df_corrupted,
                                                          self.categorical_columns,
                                                          self.numerical_columns)
                                        )
                                )
            
        
    def get_cleaned(self, df_train, df_corrupted):
        
        score_no_cleaning = self.ppp.predict_score_ppp(self.ppp_model, df_corrupted)
        print(f"PPP score no cleaning: {score_no_cleaning}")
        
        cleaner_scores_ppp = []
        for cleaner in self.cleaners:
            df_cleaned = cleaner.apply_cleaner(df_train, df_corrupted, self.categorical_columns, self.numerical_columns)
            cleaner_score = self.ppp.predict_score_ppp(self.ppp_model, df_cleaned)
            print(f"PPP score with cleaning: {cleaner}: {cleaner_score}")
            cleaner_scores_ppp.append(cleaner_score)
            
        best_cleaning_idx = pd.Series(cleaner_scores_ppp).idxmax()
        best_cleaning_score = cleaner_scores_ppp[best_cleaning_idx]
        if best_cleaning_score > score_no_cleaning:
            df_cleaned = self.cleaners[best_cleaning_idx].apply_cleaner(df_train, df_corrupted, self.categorical_columns, self.numerical_columns)
            print(f"Best cleaning method: {self.cleaners[best_cleaning_idx]}: {best_cleaning_score}")
        else:
            print("Cleaning didnt't improve the score")
            
        return df_cleaned, score_no_cleaning, cleaner_scores_ppp
    
    
    def __call__(self, df_train, df_corrupted):
        return self.get_cleaned(df_train, df_corrupted)

### Outlier Detection

In [ ]:
# detection using KNN from PyOD
outlier = PyODKNN(train_data, test_data_corrupted, categorical_columns, numerical_columms)

In [ ]:
test_data_corrupted_outliers = outlier.fit_transform(train_data, test_data_corrupted)
test_data_corrupted_outliers.head(10)

In [ ]:
# detection using Isolation Forest from PyOD
outlier_if = PyODIsolationForest(train_data, test_data_corrupted, categorical_columns, numerical_columms)

In [ ]:
test_data_corrupted_outliers_if = outlier_if.fit_transform(train_data, test_data_corrupted)
test_data_corrupted_outliers_if.head(10)

#### Preparing the outliers for imputation

In [ ]:
if "outlier" in test_data_corrupted_outliers.columns:
    print(f'Setting {test_data_corrupted_outliers["outlier"].sum()} to Nan')
    test_data_corrupted_outliers.loc[test_data_corrupted_outliers["outlier"], :] = np.nan
    test_data_corrupted_outliers = test_data_corrupted_outliers.drop('outlier', axis=1)

In [ ]:
## train_data, test_data_corrupted, 
## check values in column in the training data -> check for outliers in the same column in the corrupted data
## store .loc 
## convert those .loc for those column into nan
## impute

In [ ]:
numerical_columms

In [ ]:
test_data_corrupted

In [ ]:
from abc import abstractmethod

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from pyod.models.knn import KNN
from pyod.models.iforest import IForest


class OutlierDetection:
    
    def __init__(self, df_train, df_corrupted, categorical_columns, numerical_columns):
        
        self.df_train = df_train
        self.df_corrupted = df_corrupted
        
        self.categorical_columns = categorical_columns
        self.numerical_columns = numerical_columns
        
        
        # preprocessing pipeline for numerical columns
        transformer_numeric = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('standard_scale', StandardScaler())
        ])

        # preprocessing pipeline for categorical columns
        transformer_categorical = Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='__NA__')),
            ('one_hot_encode', OneHotEncoder(handle_unknown='ignore'))
        ])

        # preprocessor
        self.feature_transform = ColumnTransformer(transformers=[
            ('categorical_features', transformer_categorical, self.categorical_columns),
            ('numerical_features', transformer_numeric, self.numerical_columns)
        ], sparse_threshold=1.0)
        
        
        @abstractmethod
        def fit_transform(self, df_train, df_corrupted):
            pass



class NoOutlierDetection(OutlierDetection):
    
    def fit_transform(self, df_train, df_corrupted):
        df_outliers = df_corrupted.copy()
        
        return df_outliers
    
    
    def __call__(self, df_train, df_corrupted):
        return self.fit_transform(df_train, df_corrupted)


        
class PyODKNN(OutlierDetection):
    
    def fit_transform(self, df_train, df_corrupted):
        df_outliers = df_corrupted.copy()
        
        feature_transformation = self.feature_transform.fit(df_train)
        x = feature_transformation.transform(df_train).toarray()
        
        model = KNN()
        model.fit(x)
        
        xx = feature_transformation.transform(df_outliers).toarray()

        df_outliers["outlier"] = model.predict(xx) ## 0: inlier, 1: outlier
        
        return df_outliers
    
    
    def __call__(self, df_train, df_corrupted):
        return self.fit_transform(df_train, df_corrupted)

    
    
class PyODIsolationForest(OutlierDetection):
    
    def fit_transform(self, df_train, df_corrupted):
        df_outliers = df_corrupted.copy()
        
        feature_transformation = self.feature_transform.fit(df_train)
        x = feature_transformation.transform(df_train).toarray()
        
        model = IForest(contamination=0.25)
        model.fit(x)
        
        xx = feature_transformation.transform(df_outliers).toarray()

        df_outliers["outlier"] = model.predict(xx) ## 0: inlier, 1: outlier
        
        return df_outliers
    
    
    def __call__(self, df_train, df_corrupted):
        return self.fit_transform(df_train, df_corrupted)


In [ ]:
from abc import abstractmethod
import numpy as np
import pandas as pd

import datawig



class Imputation:
    
    def __init__(self, df_train, df_corrupted, categorical_columns, numerical_columns):
        self.df_train = df_train
        self.df_corrupted = df_corrupted
        
        self.categorical_columns = categorical_columns
        self.numerical_columns = numerical_columns
        
    
    @abstractmethod
    def fit_transform(self, df_train, df_corrupted):
        pass

    
    
class NoImputation(Imputation):    
    
    def __init__(self, df_train, df_corrupted, categorical_columns, numerical_columns):        
        Imputation.__init__(self, df_train, df_corrupted, categorical_columns, numerical_columns)
    
    
    def fit_transform(self, df_train, df_corrupted):
        df_imputed = df_corrupted.copy()
        return df_imputed
    
    
    def __call__(self, df_train, df_corrupted):
        return self.fit_transform(df_train, df_corrupted)
    
    
    
class MeanModeImputation(Imputation):
    
    def __init__(self, df_train, df_corrupted, categorical_columns, numerical_columns):
        self.means = {}
        self.modes = {}
    
        Imputation.__init__(self, df_train, df_corrupted, categorical_columns, numerical_columns)
    
    
    def fit_transform(self, df_train, df_corrupted):
        df_imputed = df_corrupted.copy()
        
        for col in df_train.columns:
            if col in self.numerical_columns:
                # mean imputer
                mean = np.mean(df_train[col])
                self.means[col] = mean
            elif col in self.categorical_columns:
                # mode imputer
                mode = df_train[col].value_counts().index[0]
                self.modes[col] = mode
                
                
        for col in df_corrupted.columns:
            if col in self.numerical_columns:
                # mean imputer
                df_imputed[col].fillna(self.means[col], inplace=True)
            elif col in self.categorical_columns:
                # mode imputer
                df_imputed[col].fillna(self.modes[col], inplace=True)
                
        return df_imputed
    
    
    def __call__(self, df_train, df_corrupted):
        return self.fit_transform(df_train, df_corrupted)

    

class DatawigImputation(Imputation):
    
    def __init__(self, df_train, df_corrupted, categorical_columns, numerical_columns):        
        Imputation.__init__(self, df_train, df_corrupted, categorical_columns, numerical_columns)
    
    
    def fit_transform(self, df_train, df_corrupted):
        df_imputed = df_corrupted.copy()

        for col in df_train.columns:
            if pd.api.types.is_categorical_dtype(df_train[col]):
                df_train[col] = df_train[col].astype(str)

        for col in df_corrupted.columns:
            if pd.api.types.is_categorical_dtype(df_corrupted[col]):
                df_corrupted[col] = df_corrupted[col].astype(str)


        for col in self.categorical_columns + self.numerical_columns:
            output_column = col
            input_columns = list(set(df_train.columns) - set([output_column]))

            print(f"Fitting model for column: {col}")
            model = datawig.SimpleImputer(input_columns, output_column, 'imputer_model')
            model.fit(df_train)

            df_imputed = model.predict(df_imputed)
            df_imputed[col].fillna(df_imputed[col + '_imputed'], inplace=True)
            df_imputed = df_imputed[df_corrupted.columns]

        return df_imputed
    
    
    def __call__(self, df_train, df_corrupted):
        return self.fit_transform(df_train, df_corrupted)

## Evaluation

In [ ]:
# score without cleaning
model_obj.score_on_test_data(model.predict_proba(test_data))

In [ ]:
# score with corruptions
model_obj.score_on_test_data(model.predict_proba(test_data_corrupted))

In [ ]:
# score with mean/mode imputation
model_obj.score_on_test_data(model.predict_proba(test_data_mm_imputed))

In [ ]:
# score with datawig imputation
model_obj.score_on_test_data(model.predict_proba(test_data_dw_imputed))